In [ ]:
import librosa
import numpy
from PIL import Image    
import os
import random
import soundfile
import numpy

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

DB_minimum=[]
DB_maximum=[]
name=[]
def spectrogram_image(y, sr,savename, hop_length, n_mels,number):
    
    # using librosa.stft for Fourier Transformation and then amplitude to db 
    D = librosa.amplitude_to_db(numpy.abs(librosa.stft(y, n_fft=hop_length, hop_length=hop_length)), ref=numpy.max)
    mels = D 
    print(D.min(),D.max())
    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(numpy.uint8)
    im = Image.fromarray(img)#saving numpy array as an Image
    imagename='./BusTestImages/' + str(savename) + str(number)+'.jpg'
    soundname='./BusTestSounds/' + str(savename) + str(number)+'.wav'
    DB_minimum.append(D.min())
    DB_maximum.append(D.max())
    name.append(imagename)
    #print ('Creating...' + imagename)
    
    im.save(imagename)
    soundfile.write(soundname, y, samplerate=sr)#saving the 2sec audio as a wav file 
    
    
# settings
hop_length = 511 # number of samples per time-step in spectrogram
n_mels = 256 # number of bins in spectrogram. Height of image
time_steps = 256 # number of time-steps. Width of image

#creating a file so i can save generated images
try:
    if not os.path.exists('BusTestImages'):
        os.makedirs('BusTestImages')
        os.makedirs('BusTestSounds')
except OSError:
    print ('Error: Creating directory of data')


directory = os.fsencode("E:\\BusTesting\\")    
for file in os.listdir(directory):    
    filename = os.fsdecode(file)
    #print(filename)
    if filename.endswith(".wav"):
        i=0
        path="E:\\BusTesting\\"+filename
        filename=os.path.splitext(filename)[0] #returning filename without the extension
        # load audio , without changing samplerate
        y, sr = librosa.load(path,sr=None)
        # extract a fixed length window        
        length_samples = time_steps*hop_length
        while i<=4:
            start_sample = random.randint(0,9) # starting at beginning
            start_sample=start_sample*sr
            if(start_sample+length_samples<sr*10):                
                window = y[start_sample:start_sample+length_samples]
                spectrogram_image(window,sr=sr,savename=filename,hop_length=hop_length, n_mels=n_mels,number=i)
                i+=1

In [ ]:
from random import shuffle
import glob
import sys
import cv2
import numpy as np
#import skimage.io as io
import tensorflow as tf
import matplotlib.pyplot as plt

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def load_image(addr):

    img = plt.imread(addr) #loading image from path
    img3=cv2.merge([img,img,img])#converting BW 1 channel images into BW 3 channel images
    #imgplot = plt.imshow(img)
    if img is None:
        return None
    img3 = cv2.resize(img3, (256, 256), interpolation=cv2.INTER_CUBIC)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img3
 

def createDataRecord(out_filename, addrs, labels):
    # open the TFRecords file
    writer = tf.io.TFRecordWriter(out_filename)
    #writer=tf.data.experimental.TFRecordWriter(out_filename)
    for i in range(len(addrs)):
        # print how many images are saved every 1000 images
        if not i % 1000:
            print('Train data: {}/{}'.format(i, len(addrs)))
            sys.stdout.flush()
        # Load the image
        img = load_image(addrs[i])
        label = labels[i]
        minv=DB_minimum[i]
        maxv=DB_maximum[i]
        namefile=name[i]

        if img is None:
            continue

        # Create a feature
        feature = {
            'image_raw': _bytes_feature(img.tostring()),
            'label': _int64_feature(label),
            'minimum':_int64_feature(int(minv)),
            'maximum':_int64_feature(int(maxv)),
            'filename': _bytes_feature(namefile.encode('utf-8'))
        }
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
        
    writer.close()
    sys.stdout.flush()

In [ ]:
cat_dog_train_path = 'C:\\Users\\Σεβάκ\\Desktop\\Πτυχιακη\\BusTestImages\\*.jpg'
# read addresses and labels from the 'train' folder
addrs = glob.glob(cat_dog_train_path)
labels = [0 if 'dataAtest' in addr else 1 for addr in addrs]  # 0 = Cat, 1 = Dog
print(addrs)
print(len(labels))

In [ ]:
createDataRecord('Bustest.tfrecords', addrs, labels)